In [2]:
import numpy as np
import os, sys
import json

project_root = os.path.abspath("")  # alternative
if project_root[-12:] == 'LyoSavin2023':
    base_dir = project_root
else:
    base_dir = os.path.dirname(project_root)
sys.path.append(os.path.join(base_dir, 'core'))
sys.path.append(os.path.join(base_dir, 'core/utils'))

import torch
import torch.optim as optim
import time
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

from utils import remove_all_ticks_and_labels

global device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}', flush=True)

Using device: cuda


In [32]:
from models import Net

description = {
    'model_name': 'large-noisy-image-classifier',
    'model_num': 4,
    'num_hidden': 32,
    'dataset': 'mnist',
    'num_epochs': 2e2,
    'lr': 4e-3,
    'num_steps': 150,
    'batch_size': 512,
    'num_in': 28*28,
    'schedule': 'sigmoid',
    'start': 1e-7,
    'end': 1e-1,
}
# save description
with open(os.path.join(base_dir, 'core', 'model_description', f'{description["model_name"]}_{description["model_num"]}.json'), 'w') as f:
    json.dump(description, f)

classifier = Net(norm_method='nn', num_channels=description['num_hidden'], num_steps=description['num_steps'])

In [33]:
classifier

Net(
  (conv_1): Conv2d(1, 32, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (relu): ReLU()
  (conv_2): Conv2d(32, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc_1): Linear(in_features=500, out_features=10, bias=True)
  (embed): Embedding(150, 32)
)

In [34]:
from torchvision import datasets, transforms
mnist_dir = os.path.join(base_dir, 'core', 'datasets', 'mnist')

mnist_train = datasets.MNIST(mnist_dir, train=True, download=True,
                transform=transforms.Compose([
                    transforms.ToTensor(),
                    # transforms.Normalize((0.1307,), (0.3081,))
                ]))

# mnist_train.targets
print(mnist_train[0][1])

# ------------------------------- select digits ------------------------------ #
train_loader = torch.utils.data.DataLoader(
    mnist_train,
    batch_size=description['batch_size'], shuffle=True)

5


In [35]:
# training 
from image_utils import generate_many_noisy_samples, rescale_to_neg_one_to_one
global device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}', flush=True)

def classifier_loss(model, x, labels, num_classes, num_steps, schedule, start, end):
    '''
    Calculates the loss associated with the model's prediction of the data. 
    This fn is given a batch of data with associated labels. 
    '''
    batch_size = x.shape[0]
    # x = x.reshape(batch_size, -1)
    x = rescale_to_neg_one_to_one(x)
    
    # add noise to the data
    t = torch.randint(0, num_steps, size=(batch_size,), device=device).long()
    # x_noisy = generate_many_noisy_samples(num_steps, x, t, schedule, start, end, device).reshape(batch_size, -1)
    x_noisy = generate_many_noisy_samples(num_steps, x, t, schedule, start, end, device)
    
    output = model(x_noisy, t)  # the output is from a softmax, so it produces a probability of each discrete class
    # output = model(x_noisy)  # the output is from a softmax, so it produces a probability of each discrete class
    c = torch.nn.functional.one_hot(labels, num_classes).to(torch.float)  # encodes the classes with a one hot encoding
    
    # loss = (c - output).square().mean()
    loss = torch.nn.functional.cross_entropy(output, c)
    return loss


classifier.to(device)
classifier.train()
num_epochs = int(description['num_epochs'])

optimizer = optim.Adam(classifier.parameters(), lr=description['lr'])

run_dir = os.path.join(base_dir, 'demos/runs', f'{description["model_name"]}_{description["model_num"]}')
tb = SummaryWriter(run_dir, flush_secs=1)

for epoch in tqdm(range(1, int(num_epochs) + 1), total=int(num_epochs), desc='Training model', unit='epochs', miniters=int(num_epochs)/100, maxinterval=float("inf")):
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        
        loss = classifier_loss(classifier, data, target, 10, description['num_steps'], description['schedule'], description['start'], description['end'])
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        tb.add_scalar('Loss', loss.item(), epoch+batch_idx)

print('training is over at last')

# save model weights
from utils import save_model_weights
save_model_weights(classifier, description['model_name'], description['model_num'])
tb.flush()

Using device: cuda


Training model:   0%|          | 0/200 [00:00<?, ?epochs/s]

training is over at last
model state dict saved in directory: /mnt/ceph/users/blyo1/projects/LyoSavin2023/core/saved_weights/large-noisy-image-classifier_4.pt


In [36]:
# test the model performance
from utils import load_model_weights
from image_utils import generate_many_noisy_samples
model_name = 'large-noisy-image-classifier'
model_num = 4

desc_dir = os.path.join(base_dir, 'core', 'model_description', f'{model_name}_{model_num}.json')
# load json file form desc_dir
with open(desc_dir, 'r') as f:
    description = json.load(f)

# load model weights
load_model = True

if load_model == True:
    from models import LargeNoisyImageClassifier, Net
    # model = LargeNoisyImageClassifier(num_in=description['num_in'], num_hidden=description['num_hidden'], num_classes=10, num_steps=description['num_steps'])
    model = Net(norm_method='nn', num_channels=description['num_hidden'])
    model = model.to(device)
    from utils import load_model_weights
    model_name = f"{description['model_name']}_{description['model_num']}.pt"
    loadpath = os.path.join(base_dir, f'core/saved_weights', model_name)
    state_dict = torch.load(loadpath, map_location=device)
    # model = load_model_weights(model, description['model_name'], description['model_num'], device)
    # model = model.to(device)
    model.eval()
    
# load data
def test_classifier_accuracy(model, num_test_samples,):
    import torch
    from torchvision import datasets, transforms
    mnist_dir = os.path.join(base_dir, 'core', 'datasets', 'mnist')
    mnist_test = datasets.MNIST(mnist_dir, train=False, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                    ]))

    test_loader = torch.utils.data.DataLoader(
        mnist_test,
        batch_size=num_test_samples, shuffle=True)

    # test the model
    from image_utils import get_classifier_acc

    for batch_idx, (test_data, test_targets) in enumerate(test_loader):
        test_data = test_data.to(device)
        # add noise to image
        
        test_targets = test_targets.to(device)
        acc = get_classifier_acc(model, test_data, test_targets)
        print(acc)
        break

test_classifier_accuracy(model, 100)

predictions: tensor([3, 6, 6, 3, 3, 9, 3, 3, 9, 3], device='cuda:0')
correct: tensor([6, 3, 2, 8, 4, 8, 1, 6, 2, 1], device='cuda:0')
0.03
